https://github.com/ThilinaRajapakse/simpletransformers/tree/master/examples

In [ ]:
#!pip install simpletransformers

In [ ]:
#!pip install torch

In [8]:
#!pip install tokenizers==0.9.4

In [1]:
#!pip install --ignore-installed six

In [2]:
#!pip install transformers -U

In [2]:
import numpy as np
import pandas as pd
from scipy.special import softmax

In [1]:
from simpletransformers.ner import NERModel

In [3]:
# Creating train_df  and eval_df for demonstration
train_data = [
    [0, "Simple", "B-MISC"],
    [0, "Transformers", "I-MISC"],
    [0, "started", "O"],
    [0, "with", "O"],
    [0, "text", "O"],
    [0, "classification", "B-MISC"],
    [1, "Simple", "B-MISC"],
    [1, "Transformers", "I-MISC"],
    [1, "can", "O"],
    [1, "now", "O"],
    [1, "perform", "O"],
    [1, "NER", "B-MISC"],
]

In [4]:
train_df = pd.DataFrame(train_data, columns=["sentence_id", "words", "labels"])

In [5]:
eval_data = [
    [0, "Simple", "B-MISC"],
    [0, "Transformers", "I-MISC"],
    [0, "was", "O"],
    [0, "built", "O"],
    [0, "for", "O"],
    [0, "text", "O"],
    [0, "classification", "B-MISC"],
    [1, "Simple", "B-MISC"],
    [1, "Transformers", "I-MISC"],
    [1, "then", "O"],
    [1, "expanded", "O"],
    [1, "to", "O"],
    [1, "perform", "O"],
    [1, "NER", "B-MISC"],
]

In [6]:
eval_df = pd.DataFrame(eval_data, columns=["sentence_id", "words", "labels"])

In [10]:
# Create a NERModel
model = NERModel("bert", "bert-base-cased", use_cuda=False, args={"overwrite_output_dir": True, "reprocess_input_data": True})

Downloading: 100%|██████████| 436M/436M [00:32<00:00, 13.3MB/s]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification

In [11]:
# Train the model
model.train_model(train_df)

Epoch 1 of 1: 100%|██████████| 1/1 [00:03<00:00,  3.29s/it]


(1, 2.210407018661499)

In [12]:
# Evaluate the model
result, model_outputs, predictions = model.eval_model(eval_df)

Running Evaluation: 100%|██████████| 1/1 [00:00<00:00,  1.83it/s]


In [13]:
# Predictions on arbitary text strings
sentences = ["Some arbitary sentence", "Simple Transformers sentence"]
predictions, raw_outputs = model.predict(sentences)

print(predictions)

Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  2.23it/s][[{'Some': 'I-PER'}, {'arbitary': 'B-LOC'}, {'sentence': 'I-PER'}], [{'Simple': 'I-PER'}, {'Transformers': 'I-MISC'}, {'sentence': 'I-MISC'}]]



In [14]:
# More detailed preditctions
for n, (preds, outs) in enumerate(zip(predictions, raw_outputs)):
    print("\n___________________________")
    print("Sentence: ", sentences[n])
    for pred, out in zip(preds, outs):
        key = list(pred.keys())[0]
        new_out = out[key]
        preds = list(softmax(np.mean(new_out, axis=0)))
        print(key, pred[key], preds[np.argmax(preds)], preds)


___________________________
Sentence:  Some arbitary sentence
Some I-PER 0.16811945 [0.07897804, 0.13034531, 0.12625937, 0.11895697, 0.16811945, 0.08026024, 0.08786559, 0.09299413, 0.116220914]
arbitary B-LOC 0.12736197 [0.084613964, 0.12178145, 0.12736197, 0.113595344, 0.12363647, 0.1101747, 0.0992397, 0.11738086, 0.10221564]
sentence I-PER 0.14749877 [0.0953362, 0.12476595, 0.14368318, 0.11944499, 0.14749877, 0.086047135, 0.089323476, 0.10089625, 0.09300407]

___________________________
Sentence:  Simple Transformers sentence
Simple I-PER 0.15926047 [0.07373084, 0.115635455, 0.15782577, 0.10060794, 0.15926047, 0.0822481, 0.084375665, 0.09930078, 0.12701479]
Transformers I-MISC 0.14975621 [0.07707294, 0.119259, 0.14975621, 0.113746494, 0.1296764, 0.08921113, 0.09652433, 0.10395571, 0.12079785]
sentence I-MISC 0.14957005 [0.08473355, 0.11872178, 0.14957005, 0.121434666, 0.13516234, 0.09107949, 0.102629684, 0.09492215, 0.1017462]
